In [2]:
import json

from cs336_alignment import eval_

In [12]:
records = eval_.get_records("alpaca_eval", data_dir="../data")

In [17]:
records

[{'instruction': 'What are the names of some famous actors that started their careers on Broadway?',
  'output': 'Some famous actors that started their careers on Broadway include: \n1. Hugh Jackman \n2. Meryl Streep \n3. Denzel Washington \n4. Julia Roberts \n5. Christopher Walken \n6. Anthony Rapp \n7. Audra McDonald \n8. Nathan Lane \n9. Sarah Jessica Parker \n10. Lin-Manuel Miranda',
  'generator': 'text_davinci_003',
  'dataset': 'helpful_base'},
 {'instruction': 'How did US states get their names?',
  'output': 'US states get their names from a variety of sources, including Native American tribes, Spanish explorers, British colonists, and even presidents. For example, the state of Alabama was named after the Native American tribe that lived in the area, while the state of Florida gets its name from the Spanish explorer, Ponce de Leon, who explored the area in the 1500s. Other states are named after English kings (like Virginia, named after England\'s "Virgin Queen," Queen Elizabe

In [4]:
prompts = eval_.get_prompts("alpaca_eval", records)

In [5]:
prompts

['# Instruction\nBelow is a list of conversations between a human and an AI assistant (you).\nUsers place their queries under "# Query:", and your responses are under "# Answer:".\nYou are a helpful, respectful, and honest assistant.\nYou should always answer as helpfully as possible while ensuring safety.\nYour answers should be well-structured and provide detailed information. They should also have an engaging tone.\nYour responses must not contain any fake, harmful, unethical, racist, sexist, toxic, dangerous, or illegal content, even if it may be helpful.\nYour response must be socially responsible, and thus you can reject to answer some controversial topics.\n\n# Query:\n```What are the names of some famous actors that started their careers on Broadway?```\n\n# Answer:\n```',
 '# Instruction\nBelow is a list of conversations between a human and an AI assistant (you).\nUsers place their queries under "# Query:", and your responses are under "# Answer:".\nYou are a helpful, respectf

In [6]:
responses = eval_.get_responses(prompts)

INFO 06-10 11:48:00 llm_engine.py:161] Initializing an LLM engine (v0.4.3) with config: model='meta-llama/Meta-Llama-3-8B', speculative_config=None, tokenizer='meta-llama/Meta-Llama-3-8B', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=8192, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), seed=0, served_model_name=meta-llama/Meta-Llama-3-8B)


/lfs/ampere2/0/ranjanr/micromamba/envs/a5/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


INFO 06-10 11:48:01 weight_utils.py:207] Using model weights format ['*.safetensors']
INFO 06-10 11:48:06 model_runner.py:146] Loading model weights took 14.9595 GB
INFO 06-10 11:48:10 gpu_executor.py:83] # GPU blocks: 27894, # CPU blocks: 2048
INFO 06-10 11:48:11 model_runner.py:854] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 06-10 11:48:11 model_runner.py:858] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 06-10 11:48:20 model_runner.py:924] Graph capturing finished in 9 secs.


Processed prompts: 100%|██████████████████████████████████████████████████| 805/805 [01:13<00:00, 10.91it/s, Generation Speed: 3207.97 toks/s]


In [8]:
eval_.dump_responses("alpaca_eval", records, responses, "../out/alpaca_eval_dump.json")



In [27]:
with open("../scripts/alpaca_eval_vllm_llama3_70b_fn/annotations_seed0_configs.json", "r") as f:
    annotations = json.load(f)

In [4]:
pref1 = "[\n    {'model': 'model_2', 'rank': 1},\n    {'model': 'model_1', 'rank': 2}\n]"
pref2 = "[\n    {'model': 'model_1', 'rank': 1},\n    {'model': 'model_2', 'rank': 2}\n]"

In [24]:
with open("../out/alpaca_eval_dump.json", "r") as f:
    dump_records = json.load(f)

In [25]:
dump_records

[{'instruction': 'What are the names of some famous actors that started their careers on Broadway?',
  'output': 'Broadway is a street in New York City that is known for its theaters. Some famous actors who started their careers on Broadway include:\n- Hugh Jackman, who starred in the musical "The Boy from Oz" on Broadway before becoming a Hollywood actor.\n- Idina Menzel, who starred in the musical "Rent" on Broadway before becoming a successful singer and actress.\n- Lin-Manuel Miranda, who wrote and starred in the musical "Hamilton" on Broadway before becoming a successful composer and actor.\n- Patti LuPone, who starred in the musical "Evita" on Broadway before becoming a successful singer and actress.\n- Bernadette Peters, who starred in the musical "Annie" on Broadway before becoming a successful singer and actress.\n- Audra McDonald, who starred in the musical "Carousel" on Broadway before becoming a successful singer and actress.\n- Sutton Foster, who starred in the musical "Th

In [28]:
annotations = annotations[:len(records)]

In [29]:
pref1 = "[\n    {'model': 'model_2', 'rank': 1},\n    {'model': 'model_1', 'rank': 2}\n]"
pref2 = "[\n    {'model': 'model_1', 'rank': 1},\n    {'model': 'model_2', 'rank': 2}\n]"

In [30]:
def is_win(i):
    return annotations[i]["preference"] == 2
    # ann = annotations[i]
    # dump_record = dump_records[i]
    # return (
    #     (ann["raw_completion"] == pref1 and dump_record["output"] == ann["output_1"]) or
    #     (ann["raw_completion"] == pref2 and dump_record["output"] == ann["output_2"])
    # )

In [42]:
annotations[-1]

{'instruction': 'I want to get better at networking at work',
 'output_1': "Improving networking skills can greatly enhance your professional life by opening up opportunities for advancement, collaboration, and mentorship. Here are some practical steps you can take to get better at networking at work:\n\n1. **Set Clear Goals**: Determine what you want to achieve through networking. Do you want to learn more about other departments, find a mentor, or seek new career opportunities? Setting clear goals will help you focus your efforts.\n\n2. **Start with Who You Know**: Begin by strengthening relationships with your current colleagues. Show genuine interest in their work and offer help when you can. They can introduce you to their contacts, which can expand your network.\n\n3. **Prepare Your Introduction**: Have a concise and interesting way to introduce yourself that includes your name, role, and something memorable about your work or interests. This will make you more memorable to other

In [43]:
dump_records[-1]

{'instruction': "Write a symphony concert review, discussing the orchestra's performance and overall audience experience.",
 'output': 'The symphony concert was a wonderful experience. The orchestra played beautifully, and the audience was engaged and enthusiastic. The conductor led the orchestra with skill and precision, and the musicians responded with passion and energy. The concert hall was filled with the sounds of the orchestra, and the audience was captivated by the music. The concert was a great success, and I would highly recommend it to anyone who enjoys classical music.',
 'generator': 'meta-llama/Meta-Llama-3-8B',
 'dataset': 'vicuna'}

In [73]:
processed = []
for i in range(len(annotations)):
    is_win_ = is_win(i)
    if dump_records[i]["instruction"] != annotations[i]["instruction"]:
        continue
    processed.append({
        "is_win": is_win_,
        "instruction": annotations[i]["instruction"],
        "lm_output": dump_records[i]["output"],
        "ref_output": annotations[i]["output_1" if annotations[i]["output_2"][:20] == dump_records[i]["output"][:20] else "output_2"],
    })

In [74]:
wins = list(filter(lambda r: r["is_win"], processed))

In [75]:
losses = list(filter(lambda r: not r["is_win"], processed))

In [76]:
len(wins) / (len(wins) + len(losses)) * 100

0.8368200836820083

In [77]:
len(wins)

2

In [78]:
len(losses)

237

In [79]:
import random
random.shuffle(losses)

In [80]:
for r in losses[:10]:
    print("INSTRUCTION")
    print("------------")
    print(r["instruction"])
    print()
    print("LM_OUTPUT")
    print("-------------")
    print(r["lm_output"])
    print()
    print("REF_OUTPUT")
    print("-------------")
    print(r["ref_output"])
    print()
    print("================")
    print()

INSTRUCTION
------------
What causes the northern lights?

LM_OUTPUT
-------------
The northern lights are caused by the interaction of charged particles from the sun with the Earth's magnetic field. The charged particles are carried by the solar wind, which is a stream of charged particles that flows from the sun. When these charged particles interact with the Earth's magnetic field, they are deflected towards the poles, where they collide with the Earth's atmosphere. This collision causes the charged particles to emit light, which we see as the northern lights.

REF_OUTPUT
-------------
The Northern Lights, also known as the Aurora Borealis, are caused by the interaction between the Earth's magnetic field and charged particles from the sun. Here is the process that leads to this spectacular natural phenomenon:

1. **Solar Wind**: The sun emits a stream of charged particles known as the solar wind. This stream of particles is composed mostly of electrons and protons.

2. **Earth’s Mag

In [81]:
"abc".endswith("c")

True